In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

# 예전에 만든 다나와 관심상품 목록 파싱하는 함수 재 사용
def read_danawa_wishlist():
    
    driver = webdriver.Chrome('./chromedriver.exe')
    driver.implicitly_wait(3)
    driver.get("https://www.danawa.com/")
               
        # 다나와 메인화면의 로그인 버튼을 누르는 작업 실행
    login = driver.find_element_by_css_selector('li.my_page_service > a')
    login.click()
    driver.implicitly_wait(3)
    
    # 아이디/비밀번호를 입력하고 로그인하기 버튼을 누르는 작업 실행
    my_id = "chch4934"
    my_pw = "dftn7jj7@"  
    
    driver.find_element_by_id('danawa-member-login-input-id').send_keys(my_id)
    driver.implicitly_wait(2)
    driver.find_element_by_name('password').send_keys(my_pw)
    driver.implicitly_wait(2)
    driver.find_element_by_css_selector('button.btn_login').click()
    driver.implicitly_wait(2)
    
    # 관심상품 목록 HTML 페이지 가져오기
    driver.find_element_by_css_selector('li.interest_goods_service > a').click()
    driver.implicitly_wait(2)
    html_src = driver.page_source
    driver.close()
    
    # 관심상품 목록 HTML 페이지를 BeautifulSoup으로 파싱하고, 데이터프레임으로 정리하기
    soup = BeautifulSoup(html_src, 'lxml')
    
    wish_table = soup.select('table[class="tbl wish_tbl"]')[0]
    wish_items = wish_table.select('tbody tr')
    
    
    titles=[]; prices=[]; links=[];
    
    for item in wish_items:
        title = item.find('div', {'class':'tit'}).text
        price = item.find('span', {'class':'price'}).text
        link = item.find('a', href=re.compile('prod.danawa.com/info/')).get('href')
        
        titles.append(title)
        prices.append(price)
        links.append(link)
      
        result = {'title':titles, 'price':prices, 'link':links}
    
    data = pd.DataFrame(result)
    
    return data
               
data = read_danawa_wishlist()
print(data, '\n')

# 데이터 프레임의 행 인덱스 속성값은 정수 0,1이므로, db의 id 필드에 넣어줄때는 1부터 시작할 수 있도록 기존 행 인덱스의 값에 숫자 1을 더한 배열을 id에 추가
for idx in data.index:
    data.loc[idx, 'id'] = idx + 1
    
# id 열이 추가된 데이터 프레임 출력 
print(data, '\n')

print(data.price)
data.price = data.price.str.replace('원','')
data.price = data.price.str.replace(',','')
data.price = data.price.astype('int')
print(data.price)
data.id = data.id.astype('int')
print(data.id)
print('\n')

update_db = data.set_index('id')
print(update_db, '\n')

# 데이터프레임을 DB에 업로드 (테이블 추가)
import sqlite3
import os
db_file = os.path.join(os.getcwd(), 'danawa.sqlite')
conn = sqlite3.connect(db_file)

# 데이터프레임 객체에 to_sql()메소드를 적용하면 데이터베이스의 테이블로 변환
# if_exists 옵션을 replace로 설정하면, 데이터베이스에 있는 같은 이름의 테이블이 존재하는 경우 새로운 테이블로 치환 
update_db.to_sql('Product', conn, if_exists = 'replace')

# DB 테이블 내용 확인
cur = conn.cursor()
cur.execute('SELECT * from Product')
rows = cur.fetchall()
for row in rows:
    print(row)
    
# DB 연결 종료
conn.close()

                               title                       price  \
0        아이가구 어반레스트 러블렌 가죽 리클라이너 1인용  \n294,670원\n\t\t\t\t\t\t\t   
1       클렙튼 퍼스트클래스 0002 가죽 리클라이너 1인용  \n271,310원\n\t\t\t\t\t\t\t   
2       클렙튼 퍼스트클래스 0049 가죽 리클라이너 1인용  \n218,400원\n\t\t\t\t\t\t\t   
3         상도가구 이지체어 아폴론 가죽 리클라이너 1인용  \n269,440원\n\t\t\t\t\t\t\t   
4               동서가구 코니 가죽 리클라이너 1인용  \n250,090원\n\t\t\t\t\t\t\t   
5            채우리 푹쉼 시크릿 가죽 리클라이너 1인용  \n309,000원\n\t\t\t\t\t\t\t   
6       클렙튼 퍼스트클래스 0001 가죽 리클라이너 1인용  \n250,920원\n\t\t\t\t\t\t\t   
7             썬퍼니처 7108 가죽 리클라이너 1인용  \n229,890원\n\t\t\t\t\t\t\t   
8   베이직팩토리 스코나 8032 엘리카 가죽 리클라이너 1인용  \n238,290원\n\t\t\t\t\t\t\t   
9    상도가구 이지체어 모나코 8650 가죽 리클라이너 1인용  \n307,390원\n\t\t\t\t\t\t\t   
10          수오 모에르 에어레더 가죽 리클라이너 1인용  \n248,250원\n\t\t\t\t\t\t\t   
11             라로퍼니처 리오 가죽 리클라이너 1인용  \n209,910원\n\t\t\t\t\t\t\t   
12      클렙튼 퍼스트클래스 0005 가죽 리클라이너 1인용  \n259,940원\n\t\t\t\t\t\t\t   
13    에넥스 ENNEE 루작 에어레더 가죽 리클라이너 1인용  \n216,800원